In [36]:
# Inputs
alpha_x_path = '/Volumes/BRIAN/Research/WSLAP_works/data/M0647/ver22/reconstruction/recomp_alpha_x_rad_ver22_full.fits'   # type full path
alpha_y_path = '/Volumes/BRIAN/Research/WSLAP_works/data/M0647/ver22/reconstruction/recomp_alpha_y_rad_ver22_full.fits'
alpha_size = 2048   # linear size input deflection field
pix_scale = 0.065   # arcseconds per pixel
lens_z = 0.584   # lens redshift
fid_z = 3.0   # fiducial source redshift of input deflection field
src_z = 5.9   # desired source redshift
output_path = '/Volumes/BRIAN/Research/WSLAP_works/data/M0647/ver22/reconstruction/critcurve_map_ver22_z5.9_full.fits'   # type output path

In [37]:
import numpy as np
import math
import pyfits
import itertools
from multiprocessing import Pool

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70., Om0=0.3)
from astropy import constants as const

In [38]:
def Mag_func(daxdx, daxdy, daydx, daydy):
    mu = abs(1. / ((1. - daxdx) * (1. - daydy) - daxdy * daydx))
    return mu
    
def Mag_func_star(arguments):
    # Convert mag_func([daxdx, daxdy, daydx, daydy]) to mag_func(daxdx, daxdy, daydx, daydy) call
    return Mag_func(*arguments)

def Mag(z_src, alpha_x, alpha_y, z_lens=lens_z, z_fid=fid_z):
    # compute the lensing distance ratio
    d_ls1 = cosmo.angular_diameter_distance_z1z2(z_lens, z_src)
    d_s1 = cosmo.angular_diameter_distance(z_src)
    d_ls2 = cosmo.angular_diameter_distance_z1z2(z_lens, z_fid)
    d_s2 = cosmo.angular_diameter_distance(z_fid)
    D = (d_ls1 / d_s1) / (d_ls2 / d_s2)
    
    # compute the partial derivatives of alpha_x and alpha_y
    daxdy, daxdx = D * np.gradient(alpha_x)
    daydy, daydx = D * np.gradient(alpha_y)
    daxdx = np.reshape(daxdx, (1, np.size(daxdx)))
    daxdy = np.reshape(daxdy, (1, np.size(daxdy)))
    daydx = np.reshape(daydx, (1, np.size(daydx)))
    daydy = np.reshape(daydy, (1, np.size(daydy)))
    
    pool = Pool()
    mu = pool.map(Mag_func_star, itertools.izip(daxdx, daxdy, daydx, daydy))[0]
    mu = [i.value for i in mu]
    mu = np.reshape(mu, (alpha_size, alpha_size))
    pool.close()
    pool.join()
    
    return mu

In [39]:
im = pyfits.open(alpha_x_path)
alpha_x = im[0].data / math.pi * 180. * 3600. / pix_scale
im.close()

im = pyfits.open(alpha_y_path)
alpha_y = im[0].data / math.pi * 180. * 3600. / pix_scale
im.close()

In [40]:
mu = Mag(src_z, alpha_x, alpha_y)

hdu1 = pyfits.PrimaryHDU(mu)
hdu1.writeto(output_path)

print('Complete!')

Complete!
